**INSTALL LIBRARIES**





In [1]:
!pip install langchain langchain-community langchain-text-splitters
!pip install pypdf faiss-cpu sentence-transformers

**Uploading Data **

In [2]:
from google.colab import files
uploaded = files.upload()


Saving applications_of_transformers_survey_2023.pdf to applications_of_transformers_survey_2023 (2).pdf
Saving asr_in_remote_healthcare_interpreting_pilot_study_2024.pdf to asr_in_remote_healthcare_interpreting_pilot_study_2024 (2).pdf
Saving attention_is_all_you_need_2017.pdf to attention_is_all_you_need_2017 (2).pdf
Saving clinical_speech_ai_development_tutorial_2024.pdf to clinical_speech_ai_development_tutorial_2024 (2).pdf
Saving efficient_transformers_nlp_survey_2024.pdf to efficient_transformers_nlp_survey_2024 (2).pdf
Saving interpreting_pretrained_speech_models_voice_disorders_2024.pdf to interpreting_pretrained_speech_models_voice_disorders_2024 (2).pdf
Saving transformers_in_speech_processing_survey_2025.pdf to transformers_in_speech_processing_survey_2025 (2).pdf
Saving unified_acoustic_representations_voice_pathology_screening_2025.pdf to unified_acoustic_representations_voice_pathology_screening_2025 (2).pdf


In [3]:
from langchain_community.document_loaders import PyPDFLoader
import os

documents = []

for file in os.listdir():
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file)
        documents.extend(loader.load())

print(f"Total pages loaded: {len(documents)}")


Total pages loaded: 777


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(chunks)}")


Total chunks created: 4713


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipython-input-124733854.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    chunks,
    embedding_model
)

print("Vector store created!")


Vector store created!


In [7]:
query = "How are transformers used in speech processing?"

results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\nResult {i+1}:\n")
    print(doc.page_content[:300])



Result 1:

cation of transformer models in the speech processing
field. Our survey covers more than 100 papers to cover
the recent progress.
• We provide detailed coverage of this rapidly evolving field
by categorizing the papers based on their applications
Specifically, these applications include automatic sp

Result 2:

cation of transformer models in the speech processing
field. Our survey covers more than 100 papers to cover
the recent progress.
• We provide detailed coverage of this rapidly evolving field
by categorizing the papers based on their applications
Specifically, these applications include automatic sp

Result 3:

cation of transformer models in the speech processing
field. Our survey covers more than 100 papers to cover
the recent progress.
• We provide detailed coverage of this rapidly evolving field
by categorizing the papers based on their applications
Specifically, these applications include automatic sp


In [9]:
!pip install transformers accelerate torch
!pip install langchain-huggingface


In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline


In [30]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256
)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu


In [31]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [32]:
!pip install -U langchain langchain-community langchain-text-splitters langchain-huggingface


In [ ]:
!pip install -U langchain langchain-community langchain-text-splitters langchain-huggingface


In [16]:
!pip install -U langchain langchain-community langchain-text-splitters langchain-huggingface


In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


**Adding LLM**

In [74]:
import re
def clean_text(text):
    # Fix hyphenated line breaks
    text = re.sub(r"-\n\s*", "", text)

    # Remove bullet points
    text = re.sub(r"[•◦▪]", "", text)

    # Remove table/figure references like "Table I", "Fig. 2"
    text = re.sub(r"(Table|Figure|Fig\.)\s+[IVXLC0-9]+", "", text, flags=re.IGNORECASE)

    # Normalize whitespace
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\s+", " ", text)

    # Normalize terminology
    text = text.replace("multi-modal", "multimodal")

    return text.strip()

In [75]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an academic research assistant.

Your task is to READ the context and WRITE a new answer in your own words.
Do NOT copy headings, tables, figures, or sentence fragments.
Summarize and synthesize the ideas.

Context:
{context}

Question:
{question}

Answer:
"""
)


In [76]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [77]:
def format_docs(docs, max_chars=1500):
    combined = "\n\n".join(doc.page_content for doc in docs)
    cleaned = clean_text(combined)
    return cleaned[:max_chars]



In [78]:
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)


In [79]:
answer = rag_chain.invoke("How are transformers used in speech processing?")
print(answer)


Automatic speech recognition, neural speech synthesis, speech translation, speech enhancement, multimodal applications, and spoken dialogue systems.
